In [1]:
import pandas as pd
from database import select_dataset
import numpy as np

In [2]:
# FIND WAY TO WRAP THIS CLEANLY CAUSE IT's A MESS !

aquastat = select_dataset('aquastat').dropna().drop(columns=['Area'])
aquastat = aquastat[~aquastat.ISO.isin(['USA', 'CHN', 'IND', 'RUS'])] # Not parsed !
aquastat = aquastat[aquastat.Variable != 'CI']
aquastat['Year'] = aquastat['Year'].astype(int)

evapotranspiration = select_dataset('evapotranspiration').dropna().drop_duplicates()
cropcalendar = select_dataset('cropcalendar').drop(columns=['Year'])
efficientwater = select_dataset('efficientwater')
wb_data = select_dataset('worldbank')
wb_data['Year'] = wb_data['Year'].astype(int)

MWU_df = pd.concat([aquastat, evapotranspiration, cropcalendar, efficientwater, wb_data])

In [6]:
from graphmodels.utils import df_to_data_dict

In [8]:
itemized = ['Kc', 'ICA']
data_dict = df_to_data_dict(MWU_df, itemized)
data_dict['LWU'] = 0

# Model

In [9]:
from graphmodels.graphmodel import GraphModel, converte_to_format, concatenate_graph_specs
import numpy as np
from graphmodels.validation import score_model, plot_diagnostic
from models.Sarah.model_EW import EW_models, MWU_model
from graphmodels.utils import get_X_y_from_data

In [10]:
model = EW_models['IWW_model']
X, y = get_X_y_from_data(model, data_dict)
res = model.run(X)

In [12]:
for model_name, model in EW_models.items():
    print(model_name, ': ')
    X, y = get_X_y_from_data(model, data_dict)
    res = model.run(X)
    print(score_model(model, X, y))

IWW_model : 
         r2  correlation        rmse Variable
0  0.734552     0.859789   31.102888      IWR
1  0.036750     0.657641  285.155347      IWW
2  0.603102     0.821737  153.740444      AWU
EW1_partial_model : 
         r2  correlation          rmse Variable
0  0.987684     0.993828      4.519458      TWW
1 -0.279845     0.443659  12512.483489      EW1
EW2_partial_model : 
         r2  correlation          rmse Variable
0  0.989336     0.995175   4272.565052      TFA
1 -0.404156     0.195124  57353.247805      EW2
EW1_model : 


/home/simon/anaconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


         r2  correlation         rmse Variable
0  0.734552     0.859789    31.102888      IWR
1  0.036750     0.657641   285.155347      IWW
2  0.603102     0.821737   153.740444      AWU
3  0.813129     0.904383     1.014062      MWU
4  0.708943     0.862557   168.975281      TWW
5 -1.091887     0.405643  5222.323376      EW1
EW2_model : 


/home/simon/anaconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


         r2  correlation          rmse Variable
0  0.734552     0.859789     31.102888      IWR
1  0.036750     0.657641    285.155347      IWW
2  0.603102     0.821737    153.740444      AWU
3  0.813129     0.904383      1.014062      MWU
4  0.989336     0.995175   4272.565052      TFA
5 -0.404156     0.195124  57353.247805      EW2
